In [ ]:
!pip install torch torchvision -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.

In [ ]:
import torch
import os
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [ ]:
checkpoint = torch.load('/content/model_best_checkpoint.pth (2).tar')

In [ ]:
resnet18_model = models.resnet18()
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 50
resnet18_model.fc = nn.Linear(num_ftrs, number_of_classes)
resnet18_model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [ ]:
torch.save(resnet18_model, 'best_model.pth')

In [ ]:
mean = [0.4731, 0.4613, 0.3259]
std = [0.2330, 0.2260, 0.2194]

image_transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))])

In [ ]:
def classify(model, image_transforms, path, dictionary):
  model = model.eval()
  image =  Image.open(path)
  image = image_transforms(image).float()
  image = image.unsqueeze(0)


  output = model(image)

  _, predicted = torch.max(output.data, 1)

  return dictionary[predicted.item()]


In [ ]:
clas=train_dataset.classes

In [ ]:
train_transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))])
train_dataset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/train_butterflies/train_split', transform = train_transforms)


In [ ]:
dictionary =[]
for i in clas:
  dictionary.append(int(i.split('_')[1]))

In [ ]:
import os
answ = {}
for address, dirs, files in os.walk('/content/drive/MyDrive/test_butterflies/valid'):
    for name in files:
        answ[int(name.split('.')[0])] = classify(resnet18_model, image_transforms, os.path.join(address, name), dictionary)


In [ ]:
new_answ = sorted(answ.items())
nn_answ ={}
for i in new_answ:
  nn_answ[i[0]] = i[1]

In [ ]:
import pandas as pd
df = pd.DataFrame(list(nn_answ.items()), columns=['index', 'label'])

In [ ]:
df

,index,label
0,0,14
1,1,40
2,2,8
3,3,37
4,4,27
...,...,...
245,245,32
246,246,6
247,247,2
248,248,4


In [ ]:
df.to_csv('res.csv', index=False)